# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [53]:
# your code here

# import all the necessary libraries
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine

In [54]:
# Create the connection to the database

driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'retail_sales'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [55]:
query = """
SHOW TABLES
"""

tables = pd.read_sql(query, engine)
tables

,Tables_in_retail_sales
0,raw_sales
1,sales_by_item
2,sales_by_item_index
3,sales_by_shop


In [56]:
query = """
SELECT *
FROM sales_by_shop
"""

sales_by_shop = pd.read_sql(query, engine)

In [57]:
query = """
SELECT *
FROM raw_sales
"""

raw_sales = pd.read_sql(query, engine)

In [58]:
query = """
SELECT *
FROM sales_by_item_index
"""

sales_item_index = pd.read_sql(query, engine)

In [59]:
query = """
SELECT *
FROM sales_by_item
"""

sales_by_item = pd.read_sql(query, engine)

In [60]:
## Checking the sales_by_shop table
print(sales_by_shop.head())
sales_by_shop.info()

   shop_id  shop_earnings  total_items_sold        date
0        2        33023.5              27.0  03/09/2019
1        3        22481.0              11.0  03/09/2019
2        4         9787.0              13.0  03/09/2019
3        5        11046.0              15.0  03/09/2019
4        6        38784.0              50.0  03/09/2019
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
shop_id             90 non-null int64
shop_earnings       90 non-null float64
total_items_sold    90 non-null float64
date                90 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 2.9+ KB


In [61]:
sales_by_shop = sales_by_shop.astype({'date': 'datetime64[ns]'})
sales_by_shop = sales_by_shop.astype({'total_items_sold': int})

In [62]:
## Checking the raw_sales table
print(raw_sales.head())
raw_sales.info()


        date  shop_id  item_id  item_price  item_cnt_day
0 2015-01-04       29     1469      1199.0           1.0
1 2015-01-04       28    21364       479.0           1.0
2 2015-01-04       28    21365       999.0           2.0
3 2015-01-04       28    22104       249.0           2.0
4 2015-01-04       28    22091       179.0           1.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 5 columns):
date            4545 non-null datetime64[ns]
shop_id         4545 non-null int64
item_id         4545 non-null int64
item_price      4545 non-null float64
item_cnt_day    4545 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 177.6 KB


In [63]:
raw_sales = raw_sales.astype({'item_cnt_day': int})

In [64]:
# Cheking the sales_by_item table
print(sales_by_item.head())
sales_by_item.info()

   item_id  item_earnings  total_items_sold        date
0       30          169.0               1.0  03/09/2019
1       31          363.0               1.0  03/09/2019
2       32          149.0               1.0  03/09/2019
3       42          299.0               1.0  03/09/2019
4       59          249.0               1.0  03/09/2019
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2955 entries, 0 to 2954
Data columns (total 4 columns):
item_id             2955 non-null int64
item_earnings       2955 non-null float64
total_items_sold    2955 non-null float64
date                2955 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 92.4+ KB


In [65]:
sales_by_item = sales_by_item.astype({'date': 'datetime64[ns]'})
sales_by_item = sales_by_item.astype({'total_items_sold': int})

## Agregations

In [67]:
by_store = sales_by_shop.groupby('shop_id').sum()
by_store = by_store.rename(columns = {'shop_earnings': 'earnings_per_store'})
by_store.head()

,earnings_per_store,total_items_sold
shop_id,,
2,66047.0,54
3,44962.0,22
4,19574.0,26
5,22092.0,30
6,77568.0,100


In [50]:
by_item = sales_by_item.groupby('item_id').sum()
by_item = by_item.rename(columns = {'item_earnings': 'earnings_per_item'})
by_item.head()

,earnings_per_item,total_items_sold
item_id,,
30,507.0,3
31,1089.0,3
32,447.0,3
42,897.0,3
59,747.0,3


In [51]:
by_store.to_csv('../totals_by_store.csv')

In [52]:
by_item.to_csv('../totals_by_items.csv')

In [68]:
# uploading it to my local database

driver = 'mysql+pymysql:'
user = 'root'
password = '19693903'
ip = 'localhost'
database = 'lab_sales'


In [69]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [70]:
by_store.to_sql('total_by_store',engine)

In [71]:
by_item.to_sql('total_by_item', engine)